In [0]:
print("Healthy")

In [0]:
from datetime import datetime, timedelta

from re import search
from sqlalchemy import create_engine
import duckdb

import pandas as pd 
import numpy as np 
import mysql.connector


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)


import json

import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)


Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
     host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)

host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

engine = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")


In [0]:
def load_table(schema_name, table_name, dataframe):
    sql = f'''
    DROP TABLE IF EXISTS {schema_name}.{table_name};
    '''

    with engine.begin() as con:     
        con.execute(sql)

    chunk_size = 10000
    num_rows = len(dataframe)

    conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
    num_iterations = (num_rows // chunk_size) + 1 if num_rows % chunk_size != 0 else num_rows // chunk_size

    for i in range(num_iterations):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if (i + 1) * chunk_size < num_rows else num_rows

        chunk_df = dataframe.iloc[start_idx:end_idx]

        chunk_df.to_sql(schema= schema_name, name = table_name, con =conn, index = False, if_exists = 'replace', chunksize = 5000, method = 'multi')

    print("Data loaded successfully.")
    

In [0]:
import pandas as pd
from datetime import datetime, timedelta

# Define start and end dates
start_date = datetime(2018, 1, 1)
end_date = datetime.now() - timedelta(days = 1)

# Generate list of dates
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create DataFrame
df = pd.DataFrame({'date': date_range})

# Display DataFrame
df.head()

In [0]:
import datetime
RUNDAY = 1
day = datetime.datetime.today().day
if day == RUNDAY:
    print("Healthy")
else:
    pass  

In [0]:
%%time
if day == RUNDAY:

    Retail_Customers_MoM = pd.read_sql(f'''
    SELECT left(mc.activation_date,7) AS 'Month', 
    COUNT(mc.id) as Customers, 
    NOW() AS load_date
    FROM m_savings_account msa
    LEFT JOIN m_client mc ON mc.id = msa.client_id
    LEFT JOIN m_savings_product msp ON msp.id = msa.product_id
    WHERE msa.product_id IN (29,31,33)
    AND left(mc.activation_date,7) < LEFT(CURRENT_DATE(),7)
    GROUP BY 1
    ORDER BY 1;
    ''', Mifosdb)
    Retail_Customers_MoM


    load_table('gold','Retail_Customers_MoM', Retail_Customers_MoM)    
    
else:
    pass  

In [0]:
%%time
Retail_Customers_MTD = pd.read_sql(f'''
SELECT CAST(left(mc.activation_date,10) AS DATE) AS Date, COUNT(mc.id) as Customers, NOW() AS load_date
FROM m_savings_account msa
LEFT JOIN m_client mc ON mc.id = msa.client_id
LEFT JOIN m_savings_product msp ON msp.id = msa.product_id
WHERE msa.product_id IN (29,31,33)
AND left(mc.activation_date,4) = LEFT(CURRENT_DATE(),4)
AND LEFT(mc.activation_date,10) < CURRENT_DATE()
GROUP BY 1
ORDER BY 1;
''', Mifosdb)
Retail_Customers_MTD

load_table('gold','Retail_Customers_MTD', Retail_Customers_MTD)  

In [0]:
%%time
if day == RUNDAY:

    Retail_Tier_MoM = pd.read_sql(f'''
    SELECT 
    LEFT(mc.activation_date, 7) AS Month,
    IFNULL(COUNT(case WHEN mc.client_level_cv_id = '99' then mc.id END) ,0) AS Tier_1,
    IFNULL(COUNT(case WHEN mc.client_level_cv_id = '100' then mc.id END) ,0) AS Tier_2,
    IFNULL(COUNT(case WHEN mc.client_level_cv_id = '101' then mc.id END) ,0) AS Tier_3,
    COUNT(mc.id) AS Total_Customers, 
    NOW() AS load_date
    FROM m_savings_account msa
    LEFT JOIN m_client mc ON mc.id = msa.client_id
    LEFT JOIN m_savings_product msp ON msp.id = msa.product_id
    WHERE msa.product_id IN (29,31,33)
    AND left(mc.activation_date, 7) < LEFT(NOW(), 7)
    GROUP BY 1
    ORDER BY 1;;
    ''', Mifosdb)
    Retail_Tier_MoM

    load_table('gold','Retail_Tier_MoM',Retail_Tier_MoM )    
    
else:
    pass  

In [0]:
%%time
Retail_Tier_MTD = pd.read_sql(f'''
SELECT 
CAST(LEFT(mc.activation_date, 10) AS DATE) AS Date,
IFNULL(COUNT(case WHEN mc.client_level_cv_id = '99' then mc.id END) ,0) AS Tier_1,
IFNULL(COUNT(case WHEN mc.client_level_cv_id = '100' then mc.id END) ,0) AS Tier_2,
IFNULL(COUNT(case WHEN mc.client_level_cv_id = '101' then mc.id END) ,0) AS Tier_3,
COUNT(mc.id) AS Total_Customers, 
NOW() AS load_date
FROM m_savings_account msa
LEFT JOIN m_client mc ON mc.id = msa.client_id
LEFT JOIN m_savings_product msp ON msp.id = msa.product_id
WHERE msa.product_id IN (29,31,33)
AND left(mc.activation_date, 4) < LEFT(NOW(), 4)
GROUP BY 1
ORDER BY 1;;
''', Mifosdb)
Retail_Tier_MTD


load_table('gold','Retail_Tier_MTD', Retail_Tier_MTD)  

In [0]:
%%time
Retail_Gender = pd.read_sql(f'''
select lower(gender) as Gender, count(distinct client_id) as Customers, GETDATE() AS load_date
from dwh_demographics dd 
where gender is not null
and gender != ''
group by 1
order by 1;
''', conn)
Retail_Gender

load_table('gold','Retail_Gender', Retail_Gender) 

In [0]:
%%time
if day == RUNDAY:

    WhatsApp_Onboard_MoM = pd.read_sql(f'''
    SELECT LEFT(USC.TIMESTAMP, 7) AS Month, COUNT(*) AS Customers, NOW() AS load_date
    from UM_SOCIAL_CLIENTS USC
    WHERE USC.ID > 1
    AND LEFT(USC.TIMESTAMP, 7) != LEFT(NOW(),7)
    GROUP BY 1
    ORDER BY 1;
    ''', Dumpdb)
    WhatsApp_Onboard_MoM

    load_table('gold','WhatsApp_Onboard_MoM',WhatsApp_Onboard_MoM )    
    
else:
    pass  

In [0]:
%%time
WhatsApp_Onboard_MTD = pd.read_sql(f'''
SELECT LEFT(USC.TIMESTAMP, 10) AS Date, COUNT(*) AS Customers, NOW() AS load_date
from UM_SOCIAL_CLIENTS USC
WHERE USC.ID > 1
AND LEFT(USC.TIMESTAMP, 4) = LEFT(NOW(),4)
GROUP BY 1
ORDER BY 1;
''', Dumpdb)
WhatsApp_Onboard_MTD

load_table('gold','WhatsApp_Onboard_MTD', WhatsApp_Onboard_MTD )

In [0]:
%%time
if day == RUNDAY:

    Backend_Onboard_MoM = pd.read_sql(f'''
    SELECT LEFT(mc.activation_date,7) AS Month, count(mc.id) AS 'Accounts', NOW() AS load_date
    FROM m_savings_account msa
    JOIN m_client mc ON mc.id = msa.client_id
    WHERE msa.product_id IN (29,31,33)
    AND msa.approvedon_userid != '31'
    AND LEFT(mc.activation_date,7) != LEFT(NOW(), 7)
    GROUP BY 1
    ORDER BY 1;
    ''', Mifosdb)
    Backend_Onboard_MoM

    load_table('gold','Backend_Onboard_MoM', Backend_Onboard_MoM)    
    
else:
    pass  


In [0]:
%%time

if day == RUNDAY:
    intra_mom = pd.read_sql(f'''
    select
    left(dat.transaction_date, 7) as month,
    isnull(count(case when channel IS NULL then transaction_id  end), 0) as NULL_Volume,
    isnull(count(case when channel IS NULL then transaction_id end), 0) as NULL_Volume,
    isnull(count(case when channel = 'MOBILE' then transaction_id  end), 0) as MOBILE_Volume,
    isnull(count(case when channel = 'USSD' then transaction_id end), 0) as USSD_Volume,
    isnull(count(case when channel = 'CHATBOT' then transaction_id end), 0) as CHATBOT_Volume,
    isnull(SUM(case when channel = 'MOBILE' then amount end) , 0) as MOBILE_intrabank_Value,
    isnull(SUM(case when channel = 'USSD' then amount end), 0) as USSD_intrabank_Value,
    isnull(SUM(case when channel = 'CHATBOT' then amount end), 0) as CHATBOT_intrabank_Value,
    isnull(count(*), 0) as Total_Volume,
    isnull(sum(amount), 0) as Total_Value,
    GETDATE() AS load_date
    from dwh_all_transactions dat
    where left(dat.transaction_date, 7) < LEFT(GETDATE(), 7)
    and (dat.channel in ('MOBILE', 'USSD', 'CHATBOT') OR dat.channel IS NULL)
    and dat.partners_description = 'Internal transfer'
    GROUP BY 1
    order by 1
    ''', conn)
    intra_mom


    load_table('gold','intra_mom', intra_mom)    
    
else:
    pass

In [0]:
%%time
intra_mtd = pd.read_sql(f'''
select
CAST(left(dat.transaction_date, 10) AS DATE) as Date,
isnull(count(case when channel = 'MOBILE' then transaction_id  end) , 0) as MOBILE_intrabank_Volume,
isnull(count(case when channel = 'USSD' then transaction_id end) , 0) as USSD_intrabank_Volume,
isnull(count(case when channel = 'CHATBOT' then transaction_id end) , 0) as CHATBOT_intrabank_Volume,
isnull(SUM(case when channel = 'MOBILE' then amount end) , 0) as MOBILE_intrabank_Value,
isnull(SUM(case when channel = 'USSD' then amount end), 0) as USSD_intrabank_Value,
isnull(SUM(case when channel = 'CHATBOT' then amount end), 0) as CHATBOT_intrabank_Value,
isnull(count(*), 0) as Total_Volume,
isnull(sum(amount), 0) as Total_Value,
GETDATE() AS load_date
from dwh_all_transactions dat
where left(dat.transaction_date, 4) = LEFT(GETDATE(), 4)
AND left(dat.transaction_date, 10) < cast(getdate() as date)
and dat.channel in ('MOBILE', 'USSD', 'CHATBOT')
and dat.partners_description = 'Internal transfer'
GROUP BY 1
order by 1
''', conn)
intra_mtd

load_table('gold','intra_mtd', intra_mtd)

In [0]:
%%time

if day == RUNDAY:

    inter_failure_mom = pd.read_sql(f'''
    SELECT LEFT(MBS.POSTING_DATE, 7) AS Month,
    count(case when CHANNEL = 'MOBILE' then ID  end) as MOBILE_Failure_Volume,
    count(case when CHANNEL = 'USSD' then ID end) as USSD_interbank_Volume,
    count(case when CHANNEL = 'CHATBOT' then ID end) as CHATBOT_Failure_Volume,
    SUM(case when CHANNEL = 'MOBILE' then AMOUNT end) as MOBILE_Failure_Value,
    SUM(case when CHANNEL = 'USSD' then AMOUNT end) as USSD_Failure_Value,
    SUM(case when CHANNEL = 'CHATBOT' then AMOUNT end) as CHATBOT_Failure_Value,
    count(*) as Total_Failure_Volume,
    sum(amount) as Total_Failure_Value,
    NOW() AS load_date
    FROM MM_BANK_SETTLEMENTS MBS
    WHERE LEFT(MBS.POSTING_DATE, 4) < LEFT(NOW(), 7)
    AND MBS.PARTNERS = 'NIPCREDIT'
    AND MBS.CHANNEL IN ('MOBILE', 'CHATBOT', 'USSD')
    AND MBS.RESPONSE_CODE != '00'
    GROUP BY 1
    ORDER BY 1;
    ''', Dumpdb)
    inter_failure_mom

    load_table('gold','inter_failure_mom',inter_failure_mom )    
    
else:
    pass  

In [0]:
%%time
inter_failure_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(MBS.POSTING_DATE, 10) AS Date) AS Date,
count(case when CHANNEL = 'MOBILE' then ID  end) as MOBILE_Failure_Volume,
count(case when CHANNEL = 'USSD' then ID end) as USSD_interbank_Volume,
count(case when CHANNEL = 'CHATBOT' then ID end) as CHATBOT_Failure_Volume,
SUM(case when CHANNEL = 'MOBILE' then AMOUNT end) as MOBILE_Failure_Value,
SUM(case when CHANNEL = 'USSD' then AMOUNT end) as USSD_Failure_Value,
SUM(case when CHANNEL = 'CHATBOT' then AMOUNT end) as CHATBOT_Failure_Value,
count(*) as Total_Failure_Volume,
sum(amount) as Total_Failure_Value,
NOW() AS load_date
FROM MM_BANK_SETTLEMENTS MBS
WHERE LEFT(MBS.POSTING_DATE, 4) = LEFT(NOW(), 4)
AND MBS.PARTNERS = 'NIPCREDIT'
AND MBS.CHANNEL IN ('MOBILE', 'CHATBOT', 'USSD')
AND MBS.RESPONSE_CODE != '00'
GROUP BY 1
ORDER BY 1;
''', Dumpdb)
inter_failure_mtd

load_table('gold','inter_failure_mtd',inter_failure_mtd )

In [0]:
%%time

if day == RUNDAY:

    avg_customer_trxn_inter_mom = pd.read_sql(f'''
    select left(dat.transaction_date,7) as Month,
        COUNT(DISTINCT dat.client_id) AS total_customers,
        COUNT(*) / COUNT(DISTINCT dat.client_id) AS avg_transactions_per_customer,
        GETDATE() AS load_date
    from dwh_all_transactions dat 
    where left(dat.transaction_date,7)  < LEFT(GETDATE(), 7)
    and dat.transaction_type  = 'withdrawal'
    and channel in ('MOBILE', 'USSD', 'CHATBOT')
    and response_code in ('00', '201', '200', '9000')
    group by 1
    order by 1;
                    ''', conn)
    avg_customer_trxn_inter_mom

    load_table('gold','avg_customer_trxn_inter_mom', avg_customer_trxn_inter_mom)    
    
else:
    pass  

In [0]:
%%time

avg_customer_trxn_inter_mtd = pd.read_sql(f'''
select CAST(left(dat.transaction_date,10) AS Date) as Date,
    COUNT(DISTINCT dat.client_id) AS total_customers,
    COUNT(*) / COUNT(DISTINCT dat.client_id) AS avg_transactions_per_customer,
    GETDATE() AS load_date
from dwh_all_transactions dat 
where left(dat.transaction_date,4) = LEFT(GETDATE(), 4)
and dat.transaction_type  = 'withdrawal'
and channel in ('MOBILE', 'USSD', 'CHATBOT')
and response_code in ('00', '201', '200', '9000')
group by 1
order by 1;
                ''', conn)
avg_customer_trxn_inter_mtd


load_table('gold','avg_customer_trxn_inter_mtd',avg_customer_trxn_inter_mtd )

In [0]:
%%time

if day == RUNDAY:

    free_extra_inter_mom = pd.read_sql(f'''
    with tbl as (
        select left(dat.transaction_date,7) as month, 
        dat.client_id , 
        count(*) as tra_count,
        case when count(*) < 50  then count(*) else 50 end as free_transfer,
        case when count(*) < 51  then 0 else count(*) - 50 end as additional_transfer
        from dwh_all_transactions dat 
        where left(dat.transaction_date,7) < LEFT(GETDATE(), 7)
        and dat.transaction_type  = 'withdrawal'
        and dat.partners  = 'NIPCREDIT'
        and channel in ('MOBILE', 'USSD', 'CHATBOT')
        and response_code = '00'
        group by 2,1
    )
    select month, 
    sum(tra_count) total_trf,
    sum(free_transfer) as total_free_trf,
    sum(additional_transfer) as total_additional_trf,
    GETDATE() AS load_date
    from tbl
    group by 1
    order by 1;
                    ''', conn)
    free_extra_inter_mom
    
    load_table('gold','free_extra_inter_mom',free_extra_inter_mom )    
    
else:
    pass  

In [0]:
%%time
free_extra_inter_mtd = pd.read_sql(f'''
with tbl as (
	select CAST(left(dat.transaction_date,10) AS DATE) as Date, 
	dat.client_id, 
	count(*) as tra_count,
	case when count(*) < 50  then count(*) else 50 end as free_transfer,
	case when count(*) < 51  then 0 else count(*) - 50 end as additional_transfer
	from dwh_all_transactions dat 
	where left(dat.transaction_date,4) = LEFT(GETDATE(), 4)
	and dat.transaction_type  = 'withdrawal'
	and dat.partners  = 'NIPCREDIT'
	and channel in ('MOBILE', 'USSD', 'CHATBOT')
	and response_code = '00'
	group by 2,1
)
select Date, 
sum(tra_count) total_trf,
sum(free_transfer) as total_free_trf,
sum(additional_transfer) as total_additional_trf,
GETDATE() AS load_date
from tbl
group by 1
order by 1;
                ''', conn)
free_extra_inter_mtd

load_table('gold','free_extra_inter_mtd', free_extra_inter_mtd)

In [0]:
%%time

if day == RUNDAY:

    intbank_out_mom = pd.read_sql(f'''
    select
    left(transaction_date, 7) as Month,
    isnull(count(case when channel IS NULL then transaction_id  end),0) as NULL_interbank_Volume,
    isnull(count(case when channel IS NULL then transaction_id end),0) as NULL_interbank_Volume,
    isnull(count(case when channel = 'MOBILE' then transaction_id  end),0) as MOBILE_interbank_Volume,
    isnull(count(case when channel = 'USSD' then transaction_id end),0) as USSD_interbank_Volume,
    isnull(count(case when channel = 'CHATBOT' then transaction_id end),0) as CHATBOT_interbank_Volume,
    isnull(SUM(case when channel = 'MOBILE' then amount end),0) as MOBILE_interbank_Value,
    isnull(SUM(case when channel = 'USSD' then amount end),0) as USSD_interbank_Value,
    isnull(SUM(case when channel = 'CHATBOT' then amount end),0) as CHATBOT_interbank_Value,
    isnull(count(*),0) as Total_Volume,
    isnull(sum(amount),0) as Total_Value,
    GETDATE() AS load_date
    from dwh_interbank_outflows
    where  left(transaction_date, 7) < LEFT(GETDATE(),7)
    and transaction_status = 'success'
    and (channel in ('MOBILE', 'USSD', 'CHATBOT') OR channel IS NULL)
    GROUP BY 1
    order by 1
                    ''', conn)
    intbank_out_mom


    load_table('gold','intbank_out_mom',intbank_out_mom )    
    
else:
    pass  

In [0]:
%%time
intbank_out_mtd = pd.read_sql(f'''
select
CAST(left(transaction_date, 10) AS DATE) as Date,
isnull(count(case when channel = 'MOBILE' then transaction_id  end),0) as MOBILE_interbank_Volume,
isnull(count(case when channel = 'USSD' then transaction_id end),0) as USSD_interbank_Volume,
isnull(count(case when channel = 'CHATBOT' then transaction_id end),0) as CHATBOT_interbank_Volume,
isnull(SUM(case when channel = 'MOBILE' then amount end),0) as MOBILE_interbank_Value,
isnull(SUM(case when channel = 'USSD' then amount end),0) as USSD_interbank_Value,
isnull(SUM(case when channel = 'CHATBOT' then amount end),0) as CHATBOT_interbank_Value,
isnull(count(*),0) as Total_Volume,
isnull(sum(amount),0) as Total_Value,
GETDATE() AS load_date
from dwh_interbank_outflows
where  left(transaction_date, 4) = LEFT(GETDATE(),4)
and transaction_status = 'success'
and channel in ('MOBILE', 'USSD', 'CHATBOT')
GROUP BY 1
order by 1
                ''', conn)
intbank_out_mtd

load_table('gold','intbank_out_mtd', intbank_out_mtd)

In [0]:
%%time
if day == RUNDAY:
    intbank_out_failed_mom = pd.read_sql(f'''
    SELECT LEFT(MBS.POSTING_DATE, 7) AS Month,
    IFNULL(count(case when CHANNEL = 'MOBILE' then ID  end) ,0) as MOBILE_interbank_Failure_Volume,
    IFNULL(count(case when CHANNEL = 'USSD' then ID end) ,0) as USSD_interbank_Failure_Volume,
    IFNULL(count(case when CHANNEL = 'CHATBOT' then ID end) ,0) as CHATBOT_interbank_Failure_Volume,
    IFNULL(SUM(case when CHANNEL = 'MOBILE' then AMOUNT end) ,0) as MOBILE_interbank_Failure_Value,
    IFNULL(SUM(case when CHANNEL = 'USSD' then AMOUNT end) ,0) as USSD_interbank_Failure_Value,
    IFNULL(SUM(case when CHANNEL = 'CHATBOT' then AMOUNT end) ,0) as CHATBOT_interbank_Failure_Value,
    IFNULL(count(*) ,0) as Total_Failure_Volume,
    IFNULL(sum(amount) ,0) as Total_Failure_Value,
    NOW() AS load_date
    FROM MM_BANK_SETTLEMENTS MBS
    WHERE LEFT(MBS.POSTING_DATE, 7) < LEFT(CURRENT_DATE(), 7)
    AND MBS.PARTNERS = 'NIPCREDIT'
    AND MBS.CHANNEL IN ('MOBILE', 'CHATBOT', 'USSD')
    AND MBS.RESPONSE_CODE != '00'
    GROUP BY 1
    ORDER BY 1;
                    ''', Dumpdb)
    intbank_out_failed_mom


    load_table('gold','intbank_out_failed_mom',intbank_out_failed_mom )    
    
else:
    pass  

In [0]:
%%time
intbank_out_failed_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(MBS.POSTING_DATE, 10) AS DATE) AS Date,
IFNULL(count(case when CHANNEL = 'MOBILE' then ID  end) ,0) as MOBILE_interbank_Failure_Volume,
IFNULL(count(case when CHANNEL = 'USSD' then ID end) ,0) as USSD_interbank_Failure_Volume,
IFNULL(count(case when CHANNEL = 'CHATBOT' then ID end) ,0) as CHATBOT_interbank_Failure_Volume,
IFNULL(SUM(case when CHANNEL = 'MOBILE' then AMOUNT end) ,0) as MOBILE_interbank_Failure_Value,
IFNULL(SUM(case when CHANNEL = 'USSD' then AMOUNT end) ,0) as USSD_interbank_Failure_Value,
IFNULL(SUM(case when CHANNEL = 'CHATBOT' then AMOUNT end) ,0) as CHATBOT_interbank_Failure_Value,
IFNULL(count(*) ,0) as Total_Failure_Volume,
IFNULL(sum(amount) ,0) as Total_Failure_Value,
NOW() AS load_date
FROM MM_BANK_SETTLEMENTS MBS
WHERE LEFT(MBS.POSTING_DATE, 4) = LEFT(CURRENT_DATE(), 4)
AND MBS.PARTNERS = 'NIPCREDIT'
AND MBS.CHANNEL IN ('MOBILE', 'CHATBOT', 'USSD')
AND MBS.RESPONSE_CODE != '00'
GROUP BY 1
ORDER BY 1;
                ''', Dumpdb)
intbank_out_failed_mtd

load_table('gold','intbank_out_failed_mtd', intbank_out_failed_mtd)

In [0]:
%%time
if day == RUNDAY:
    
    bil_mom = pd.read_sql(f'''
    select
    left(transaction_date, 7) as Month,
    ISNULL(count(case when channel = 'MOBILE' then id end) ,0) as MOBILE_Bills_Volume,
    ISNULL(count(case when channel = 'USSD' then id end) ,0) as USSD_Bills_Volume,
    ISNULL(count(case when channel = 'CHATBOT' then id end) ,0) as CHATBOT_Bills_Volume,
    ISNULL(SUM(case when channel = 'MOBILE' then amount end) ,0) as MOBILE_Bills_Value,
    ISNULL(SUM(case when channel = 'USSD' then amount end) ,0) as USSD_Bills_Value,
    ISNULL(SUM(case when channel = 'CHATBOT' then amount end) ,0) as CHATBOT_Bills_Value,
    ISNULL(count(*) ,0) as Total_Volume,
    ISNULL(sum(amount) ,0) as Total_Value,
    GETDATE() AS load_date
    from dwh_bills_only
    where  left(transaction_date, 7) < LEFT(GETDATE(), 7)
    and transaction_status = 'success'
    and channel in ('MOBILE', 'USSD', 'CHATBOT')
    GROUP BY 1
    order by 1
                    ''', conn)
    bil_mom
    load_table('gold','bil_mom',bil_mom )    
    
else:
    pass  

In [0]:
%%time
bil_mtd = pd.read_sql(f'''
select
CAST(left(transaction_date, 10) AS DATE) as Date,
ISNULL(count(case when channel = 'MOBILE' then id end) ,0) as MOBILE_Bills_Volume,
ISNULL(count(case when channel = 'USSD' then id end) ,0) as USSD_Bills_Volume,
ISNULL(count(case when channel = 'CHATBOT' then id end) ,0) as CHATBOT_Bills_Volume,
ISNULL(SUM(case when channel = 'MOBILE' then amount end) ,0) as MOBILE_Bills_Value,
ISNULL(SUM(case when channel = 'USSD' then amount end) ,0) as USSD_Bills_Value,
ISNULL(SUM(case when channel = 'CHATBOT' then amount end) ,0) as CHATBOT_Bills_Value,
ISNULL(count(*) ,0) as Total_Volume,
ISNULL(sum(amount) ,0) as Total_Value,
GETDATE() AS load_date
from dwh_bills_only
where  left(transaction_date, 4) = LEFT(GETDATE(), 4)
and transaction_status = 'success'
and channel in ('MOBILE', 'USSD', 'CHATBOT')
GROUP BY 1
order by 1
                ''', conn)
bil_mtd
load_table('gold','bil_mtd',bil_mtd)

In [0]:
%%time
if day == RUNDAY:

    top_biller_mom = pd.read_sql(f'''
    WITH monthly_counts AS (
        SELECT
            LEFT(transaction_date, 7) AS month,
            biller,
            COUNT(*) AS biller_count
        FROM
            dwh_bills_only
            where left(transaction_date, 7) < left(getdate(),7)
            and transaction_status  = 'success'
        GROUP BY
            month,
            biller
    )

    SELECT
        month,
        biller,
        biller_count,
        getdate() as load_date
    FROM (
        SELECT
            month,
            biller,
            biller_count,
            RANK() OVER (PARTITION BY month ORDER BY biller_count DESC) AS ranking
        FROM
            monthly_counts
    ) ranked_counts
    WHERE
        ranking = 1
    ORDER BY
        month, ranking;
                    ''', conn)
    top_biller_mom

    load_table('gold','top_biller_mom', top_biller_mom)    
    
else:
    pass 

In [0]:
%%time
top_biller_mtd = pd.read_sql(f'''
WITH daily_counts AS (
    SELECT
        DATE_TRUNC('day', transaction_date) AS date,
        biller,
        COUNT(*) AS biller_count
    FROM
        dwh_bills_only
        where left(transaction_date, 4) = LEFT(GETDATE(), 4)
        and left(transaction_date, 10) < cast(getdate() as date)
        and transaction_status  = 'success'
    GROUP BY
        date,
        biller
)

SELECT
    date,
    biller,
    biller_count
FROM (
    SELECT
        date,
        biller,
        biller_count,
        RANK() OVER (PARTITION BY date ORDER BY biller_count DESC) AS ranking
    FROM
        daily_counts
) ranked_counts
WHERE
    ranking = 1
ORDER BY
    date, ranking;
                ''', conn)
top_biller_mtd

load_table('gold','top_biller_mtd', top_biller_mtd)

In [0]:
%%time
if day == RUNDAY:

    bills_mom = pd.read_sql(f'''
        SELECT
            LEFT(transaction_date, 7) AS month,
            biller,
            COUNT(*) AS biller_count
        FROM
            dwh_bills_only
            where left(transaction_date, 7) < left(getdate(),7)
            and transaction_status  = 'success'
        GROUP BY 1,2
        order by 1,2;
                    ''', conn)
    bills_mom

    load_table('gold','bills_mom', bills_mom)    
    
else:
    pass  


In [0]:
%%time
bills_mtd = pd.read_sql(f'''
    SELECT
        CAST(LEFT(transaction_date, 10) AS DATE) AS Date,
        biller,
        COUNT(*) AS biller_count
    FROM
        dwh_bills_only
        where left(transaction_date, 4) = left(getdate(),4)
        and transaction_status  = 'success'
    GROUP BY 1,2
    order by 1,2;
                ''', conn)
bills_mtd
load_table('gold','bills_mtd', bills_mtd)

In [0]:
%%time
if day == RUNDAY:
    airtime_mom = pd.read_sql(f'''
        SELECT
            LEFT(transaction_date, 7) AS month,
            biller,
            COUNT(*) AS biller_count
        FROM
            dwh_airtime_only dao 
            where left(transaction_date, 7) < left(getdate(),7)
            and transaction_status  = 'success'
        GROUP BY 1,2
        order by 1,2;

                    ''', conn)
    airtime_mom

    load_table('gold','airtime_mom',airtime_mom )    
    
else:
    pass  

In [0]:
%%time
airtime_mtd = pd.read_sql(f'''
    SELECT
        CAST(LEFT(transaction_date, 10) AS DATE) AS Date,
        biller,
        COUNT(*) AS biller_count
    FROM
        dwh_airtime_only dao 
        where left(transaction_date, 4) = left(getdate(),4)
        and transaction_status  = 'success'
    GROUP BY 1,2
    order by 1,2;

                ''', conn)
airtime_mtd
load_table('gold','airtime_mtd', airtime_mtd)

In [0]:
%%time
purchased_bill = pd.read_sql(f'''
select count(distinct client_id) as Purchase_Bill_Customers, GETDATE() AS Load_date
from dwh_bills_only;
                ''', conn)
purchased_bill

load_table('gold','purchased_bill',purchased_bill )

In [0]:
%%time
if day == RUNDAY:

    airt_mom = pd.read_sql(f'''
    select
    left(transaction_date, 7) as month,
    ISNULL(count(case when channel = 'MOBILE' then id end) , 0) as MOBILE_Airtime_Volume,
    ISNULL(count(case when channel = 'USSD' then id end) , 0) as USSD_Airtime_Volume,
    ISNULL(count(case when channel = 'CHATBOT' then id end) , 0) as CHATBOT_Airtime_Volume,
    ISNULL(SUM(case when channel = 'MOBILE' then amount end) , 0) as MOBILE_Airtime_Value,
    ISNULL(SUM(case when channel = 'USSD' then amount end) , 0) as USSD_Airtime_Value,
    ISNULL(SUM(case when channel = 'CHATBOT' then amount end) , 0) as CHATBOT_Airtime_Value,
    ISNULL(count(*) , 0) as Total_Volume,
    ISNULL(sum(amount), 0) as Total_Value,
    GETDATE() AS load_date
    from dwh_airtime_only
    where  left(transaction_date, 7) < LEFT(GETDATE(), 7)
    and transaction_status = 'success'
    and channel in ('MOBILE', 'USSD', 'CHATBOT')
    GROUP BY 1
    order by 1
                    ''', conn)
    airt_mom

    load_table('gold','airt_mom', airt_mom)    
    
else:
    pass  

In [0]:
%%time
airt_mtd = pd.read_sql(f'''
select
CAST(left(transaction_date, 10) AS DATE) as Date,
ISNULL(count(case when channel = 'MOBILE' then id end) , 0) as MOBILE_Airtime_Volume,
ISNULL(count(case when channel = 'USSD' then id end) , 0) as USSD_Airtime_Volume,
ISNULL(count(case when channel = 'CHATBOT' then id end) , 0) as CHATBOT_Airtime_Volume,
ISNULL(SUM(case when channel = 'MOBILE' then amount end) , 0) as MOBILE_Airtime_Value,
ISNULL(SUM(case when channel = 'USSD' then amount end) , 0) as USSD_Airtime_Value,
ISNULL(SUM(case when channel = 'CHATBOT' then amount end) , 0) as CHATBOT_Airtime_Value,
ISNULL(count(*) , 0) as Total_Volume,
ISNULL(sum(amount), 0) as Total_Value,
GETDATE() AS load_date
from dwh_airtime_only
where  left(transaction_date, 4) = LEFT(GETDATE(), 4)
and transaction_status = 'success'
and channel in ('MOBILE', 'USSD', 'CHATBOT')
GROUP BY 1
order by 1
                ''', conn)
airt_mtd
load_table('gold','airt_mtd',airt_mtd )

In [0]:
%%time
if day == RUNDAY:
    top_airt_mom = pd.read_sql(f'''
        WITH monthly_counts AS (
            SELECT
                DATE_TRUNC('month', transaction_date) AS month,
                biller,
                COUNT(*) AS biller_count
            FROM
                dwh_airtime_only
                where left(transaction_date, 7) < LEFT(GETDATE(), 7)
                and transaction_status  = 'success'
            GROUP BY
                month,
                biller
        )

        SELECT
            month,
            biller,
            biller_count,
            GETDATE() AS load_date
        FROM (
            SELECT
                month,
                biller,
                biller_count,
                RANK() OVER (PARTITION BY month ORDER BY biller_count DESC) AS ranking
            FROM
                monthly_counts
        ) ranked_counts
        WHERE
            ranking = 1
        ORDER BY
            month, ranking;
                        ''', conn)
    top_airt_mom

    load_table('gold','top_airt_mom', top_airt_mom)    
    
else:
    pass 

In [0]:
%%time
top_airt_mtd = pd.read_sql(f'''
WITH daily_counts AS (
    SELECT
        DATE_TRUNC('day', transaction_date) AS Date,
        biller,
        COUNT(*) AS biller_count
    FROM
        dwh_airtime_only
        where left(transaction_date, 4) = LEFT(getdate() ,4)
        and transaction_status  = 'success'
    GROUP BY
        Date,
        biller
)

SELECT
    Date,
    biller,
    biller_count,
    GETDATE() AS load_date
FROM (
    SELECT
        Date,
        biller,
        biller_count,
        RANK() OVER (PARTITION BY Date ORDER BY biller_count DESC) AS ranking
    FROM
        daily_counts
) ranked_counts
WHERE
    ranking = 1
ORDER BY
    Date, ranking;
                ''', conn)
top_airt_mtd

load_table('gold','top_airt_mtd', top_airt_mtd)

In [0]:
%%time
purchased_airtime = pd.read_sql(f'''
select count(distinct client_id) as Purchase_Airtime, GETDATE() AS Load_date
from dwh_airtime_only;
                ''', conn)
purchased_airtime
load_table('gold','purchased_airtime', purchased_airtime)

In [0]:
%%time
if day == RUNDAY:
    df_card_mom = pd.read_sql(f'''
    SELECT LEFT(TVT.LOCAL_TRANSACTION_TIME,7) AS Month, 
    IFNULL(COUNT(CASE WHEN TVT.TRANSACTION_TYPE = 'cashWithdrawal'  THEN TRANSACTION_TYPE END) ,0) AS ATM_Wdl_Volume,
    IFNULL(SUM(CASE WHEN TVT.TRANSACTION_TYPE = 'cashWithdrawal'  THEN TVT.AMOUNT END) ,0) AS ATM_Wdl_Value,
    IFNULL(COUNT(CASE WHEN TVT.TRANSACTION_TYPE = 'debitGoodsAndServices'  THEN TRANSACTION_TYPE END) ,0) AS POS_Volume,
    IFNULL(SUM(CASE WHEN TVT.TRANSACTION_TYPE = 'debitGoodsAndServices'  THEN TVT.AMOUNT END),0) AS POS_Value,
    IFNULL(COUNT(CASE WHEN TVT.TRANSACTION_TYPE = 'paymentFromAccount' THEN TRANSACTION_TYPE END),0) AS Web_payment_volume,
    IFNULL(SUM(CASE WHEN TVT.TRANSACTION_TYPE = 'paymentFromAccount'  THEN TVT.AMOUNT END),0) AS Web_payment_Value,
    IFNULL(COUNT(CASE WHEN TVT.TRANSACTION_TYPE = 'accountTransfer'  THEN TRANSACTION_TYPE END),0) AS ATM_Transfer_volume,
    IFNULL(SUM(CASE WHEN TVT.TRANSACTION_TYPE = 'accountTransfer'   THEN TVT.AMOUNT END),0) AS ATM_Transfer_Value,
    IFNULL(COUNT(*),0) AS Total_Cards_Volume, 
    IFNULL(SUM(TVT.AMOUNT),0) AS Total_Card_Value,
    NOW() AS load_date
    FROM TM_VERVE_TRANSACTIONS TVT 
    WHERE TVT.TRANSACTION_STATUS = 'success'
    AND LEFT(TVT.LOCAL_TRANSACTION_TIME,7) < LEFT(NOW(), 7)
    GROUP BY 1
    ORDER BY 1;
                    ''', Dumpdb)
    df_card_mom


    load_table('gold','df_card_mom', df_card_mom)    
    
else:
    pass

In [0]:
%%time
df_card_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(TVT.LOCAL_TRANSACTION_TIME,10) AS DATE) AS Date, 
IFNULL(COUNT(CASE WHEN TVT.TRANSACTION_TYPE = 'cashWithdrawal'  THEN TRANSACTION_TYPE END) ,0) AS ATM_Wdl_Volume,
IFNULL(SUM(CASE WHEN TVT.TRANSACTION_TYPE = 'cashWithdrawal'  THEN TVT.AMOUNT END) ,0) AS ATM_Wdl_Value,
IFNULL(COUNT(CASE WHEN TVT.TRANSACTION_TYPE = 'debitGoodsAndServices'  THEN TRANSACTION_TYPE END) ,0) AS POS_Volume,
IFNULL(SUM(CASE WHEN TVT.TRANSACTION_TYPE = 'debitGoodsAndServices'  THEN TVT.AMOUNT END),0) AS POS_Value,
IFNULL(COUNT(CASE WHEN TVT.TRANSACTION_TYPE = 'paymentFromAccount' THEN TRANSACTION_TYPE END),0) AS Web_payment_volume,
IFNULL(SUM(CASE WHEN TVT.TRANSACTION_TYPE = 'paymentFromAccount'  THEN TVT.AMOUNT END),0) AS Web_payment_Value,
IFNULL(COUNT(CASE WHEN TVT.TRANSACTION_TYPE = 'accountTransfer'  THEN TRANSACTION_TYPE END),0) AS ATM_Transfer_volume,
IFNULL(SUM(CASE WHEN TVT.TRANSACTION_TYPE = 'accountTransfer'   THEN TVT.AMOUNT END),0) AS ATM_Transfer_Value,
IFNULL(COUNT(*),0) AS Total_Cards_Volume, 
IFNULL(SUM(TVT.AMOUNT),0) AS Total_Card_Value,
NOW() AS load_date
FROM TM_VERVE_TRANSACTIONS TVT 
WHERE TVT.TRANSACTION_STATUS = 'success'
AND LEFT(TVT.LOCAL_TRANSACTION_TIME,4) = LEFT(NOW(), 4)
GROUP BY 1
ORDER BY 1;
                ''', Dumpdb)
df_card_mtd

load_table('gold','df_card_mtd', df_card_mtd)

In [0]:
%%time
if day == RUNDAY:
    phy_crd_issue_mom = pd.read_sql(f'''
    SELECT LEFT(CPAM.DATE_CREATED, 7) AS Month, 
    COUNT(*) AS Physical_Card_Issuance,
    NOW() AS load_date
    FROM CM_PAN_ACCOUNT_MAPPER CPAM
    WHERE LEFT(CPAM.DATE_CREATED, 7) < LEFT(NOW(),7)
    GROUP BY 1
    ORDER BY 1;
                    ''', Dumpdb)
    phy_crd_issue_mom

    load_table('gold','phy_crd_issue_mom', phy_crd_issue_mom)    
    
else:
    pass  

In [0]:
%%time
phy_crd_issue_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(CPAM.DATE_CREATED, 10) AS DATE) AS Date, 
COUNT(*) AS Physical_Card_Issuance,
NOW() AS load_date
FROM CM_PAN_ACCOUNT_MAPPER CPAM
WHERE LEFT(CPAM.DATE_CREATED, 4) = LEFT(NOW(),4)
GROUP BY 1
ORDER BY 1;
                ''', Dumpdb)
phy_crd_issue_mtd
load_table('gold','phy_crd_issue_mtd', phy_crd_issue_mtd)

In [0]:
%%time
if day == RUNDAY:
    card_status_mom = pd.read_sql(f'''
    SELECT LEFT(CVD.DATE_CREATED, 7) AS Month,
    IFNULL(COUNT(case when CVD.PAN_TYPE = 'PHYSICAL' THEN CVD.ID END) ,0) AS Total_Physical,
    IFNULL(COUNT(case when CVD.PAN_TYPE = 'VIRTUAL' THEN CVD.ID END) ,0) AS Total_Virtual,
    IFNULL(COUNT(case when CVD.C_STATUS = 'ACTIVE' THEN CVD.ID END) ,0) AS Total_ACTIVE,
    IFNULL(COUNT(case when CVD.C_STATUS = 'EXPIRED' THEN CVD.ID END) ,0) AS Total_EXPIRED,
    IFNULL(COUNT(case when CVD.C_STATUS = 'BLOCKED' THEN CVD.ID END) ,0) AS Total_BLOCKED,
    NOW() AS load_date
    FROM CM_VERVE_DETAILS CVD
    WHERE LEFT(CVD.DATE_CREATED, 7) < LEFT(NOW(), 7)
    GROUP BY 1
    ORDER BY 1;
                    ''', Dumpdb)
    card_status_mom

    load_table('gold','card_status_mom', card_status_mom)    
    
else:
    pass  

In [0]:
%%time
card_status_mtd = pd.read_sql(f'''
SELECT 
CAST(LEFT(CVD.DATE_CREATED, 10) AS DATE) AS Date,
IFNULL(COUNT(case when CVD.PAN_TYPE = 'PHYSICAL' THEN CVD.ID END) ,0) AS Total_Physical,
IFNULL(COUNT(case when CVD.PAN_TYPE = 'VIRTUAL' THEN CVD.ID END) ,0) AS Total_Virtual,
IFNULL(COUNT(case when CVD.C_STATUS = 'ACTIVE' THEN CVD.ID END) ,0) AS Total_ACTIVE,
IFNULL(COUNT(case when CVD.C_STATUS = 'EXPIRED' THEN CVD.ID END) ,0) AS Total_EXPIRED,
IFNULL(COUNT(case when CVD.C_STATUS = 'BLOCKED' THEN CVD.ID END) ,0) AS Total_BLOCKED,
NOW() AS load_date
FROM CM_VERVE_DETAILS CVD
WHERE LEFT(CVD.DATE_CREATED, 4) = LEFT(NOW(), 4)
GROUP BY 1
ORDER BY 1;
                ''', Dumpdb)
card_status_mtd
load_table('gold','card_status_mtd', card_status_mtd)

In [0]:
%%time
card_to_expire = pd.read_sql(f'''
SELECT CAST(CVD.EXPIRY_DATE AS SIGNED) as EXPIRY_DATE, 
COUNT(*) AS Cards_To_Expire,
NOW() AS load_date
FROM CM_VERVE_DETAILS CVD
WHERE left(CVD.EXPIRY_DATE,2) = SUBSTRING(CURRENT_DATE(), 3,2)
GROUP BY 1
ORDER BY 1;
                ''', Dumpdb)
card_to_expire
load_table('gold','card_to_expire', card_to_expire)

In [0]:
%%time
if day == RUNDAY:
    Retail_Accounts_MoM = pd.read_sql(f'''
    SELECT LEFT(msa.approvedon_date,7) AS Month, 
    msp.name AS Account_Product, 
    COUNT(msa.id) AS No_Accounts,
    NOW() as load_date
    FROM m_savings_account msa
    LEFT JOIN m_client mc ON mc.id = msa.client_id
    LEFT JOIN m_savings_product msp ON msp.id = msa.product_id
    WHERE msa.product_id IN (24,30,33)
    AND LEFT(msa.approvedon_date,7) < LEFT(NOW(),7)
    GROUP BY 2,1
    ORDER BY 2,1;
    ''', Mifosdb)
    Retail_Accounts_MoM


    load_table('gold','Retail_Accounts_MoM', Retail_Accounts_MoM)    
    
else:
    pass  

In [0]:
%%time
Retail_Accounts_MTD = pd.read_sql(f'''
SELECT CAST(LEFT(msa.approvedon_date,10) AS DATE) AS Date, 
msp.name AS Account_Product, 
COUNT(msa.id) AS No_Accounts,
NOW() as load_date
FROM m_savings_account msa
LEFT JOIN m_client mc ON mc.id = msa.client_id
LEFT JOIN m_savings_product msp ON msp.id = msa.product_id
WHERE msa.product_id IN (24,30,33)
AND LEFT(msa.approvedon_date,4) < LEFT(NOW(),4)
GROUP BY 2,1
ORDER BY 2,1;
''', Mifosdb)
Retail_Accounts_MTD

load_table('gold','Retail_Accounts_MTD', Retail_Accounts_MTD)

In [0]:
%%time
if day == RUNDAY:
    lux_pos_trxn_mom = pd.read_sql(f'''
    SELECT  
    LEFT(LMT.TRANSACTION_TIME, 7) AS Month,
    COUNT(case when LMT.STATUS_CODE = '00' then LMT.ID END) AS POS_Success_Volume,
    COUNT(case when LMT.STATUS_CODE != '00' then LMT.ID END) AS POS_Failed_Volume,
    SUM( case when LMT.STATUS_CODE = '00' then LMT.AMOUNT END) AS POS_Success_Value,
    SUM( case when LMT.STATUS_CODE != '00' then LMT.AMOUNT END) AS POS_Failed_Value,
    NOW() AS load_date
    FROM LUX_MERCHANT_TRANSACTIONS LMT
    WHERE LEFT(LMT.TRANSACTION_TIME, 7) < LEFT(NOW(), 7)
    group BY 1
    ORDER BY 1;
                    ''', Dumpdb)
    lux_pos_trxn_mom


    load_table('gold','lux_pos_trxn_mom', lux_pos_trxn_mom)    
    
else:
    pass 

In [0]:
%%time
lux_pos_trxn_mtd = pd.read_sql(f'''
SELECT  
LEFT(LMT.TRANSACTION_TIME, 10) AS Date,
COUNT(case when LMT.STATUS_CODE = '00' then LMT.ID END) AS POS_Success_Volume,
COUNT(case when LMT.STATUS_CODE != '00' then LMT.ID END) AS POS_Failed_Volume,
SUM( case when LMT.STATUS_CODE = '00' then LMT.AMOUNT END) AS POS_Success_Value,
SUM( case when LMT.STATUS_CODE != '00' then LMT.AMOUNT END) AS POS_Failed_Value,
NOW() AS load_date
FROM LUX_MERCHANT_TRANSACTIONS LMT
WHERE LEFT(LMT.TRANSACTION_TIME, 4) = LEFT(NOW(), 4)
group BY 1
ORDER BY 1;
                ''', Dumpdb)
lux_pos_trxn_mtd
load_table('gold','lux_pos_trxn_mtd', lux_pos_trxn_mtd)

In [0]:
%%time
if day == RUNDAY:
    vpay_trxn_mom = pd.read_sql(f'''
    SELECT 
    left(tw.TIME, 7) AS Month,
    count(tw.TRANSACTION_ID) as Vpay_Volume,
    sum(tw.AMOUNT) AS Vpay_Value,
    NOW() AS load_date
    FROM TM_WALLET_TRANSACTIONS tw
    WHERE tw.WALLET_NAME LIKE '%vpos%'
    and left(tw.TIME, 7) < LEFT(NOW(), 7)
    GROUP BY 1
    ORDER BY 1;
                    ''', Dumpdb)
    vpay_trxn_mom

    load_table('gold','vpay_trxn_mom', vpay_trxn_mom)    
    
else:
    pass  

In [0]:
%%time
vpay_trxn_mtd = pd.read_sql(f'''
SELECT 
left(tw.TIME, 10) AS Month,
count(tw.TRANSACTION_ID) as Vpay_Volume,
sum(tw.AMOUNT) AS Vpay_Value,
NOW() AS load_date
FROM TM_WALLET_TRANSACTIONS tw
WHERE tw.WALLET_NAME LIKE '%vpos%'
and left(tw.TIME, 4) = LEFT(NOW(), 4)
GROUP BY 1
ORDER BY 1;
                ''', Dumpdb)
vpay_trxn_mtd
load_table('gold','vpay_trxn_mtd', vpay_trxn_mtd)

In [0]:
%%time
if day == RUNDAY:
    fd_corporate_mom = pd.read_sql(f'''
    SELECT LEFT(st.transaction_date, 7) AS Month, 
    sum(st.amount) AS Corporate_FD_Amount,
    NOW() AS load_date
    from m_savings_account_transaction st
    inner join m_savings_account msa on msa.id =st.savings_account_id
    inner join m_savings_product msp on msp.id=msa.product_id
    left join m_client cl on cl.id=msa.client_id
    where st.is_reversed=0
    AND msa.product_id = '36'
    AND st.transaction_type_enum = 1
    and LEFT(st.transaction_date, 7) < LEFT(NOW(), 7)
    AND cl.id IN (
        SELECT DISTINCT msa.client_id
        FROM m_savings_account msa
        WHERE msa.product_id = '26'
    )
    GROUP BY 1
    ORDER BY 1;
                    ''', Mifosdb)
    fd_corporate_mom


    load_table('gold','fd_corporate_mom', fd_corporate_mom)    
    
else:
    pass 

In [0]:
%%time
fd_corporate_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(st.transaction_date, 10) AS DATE) AS Date, 
sum(st.amount) AS Corporate_FD_Amount,
NOW() AS load_date
from m_savings_account_transaction st
inner join m_savings_account msa on msa.id =st.savings_account_id
inner join m_savings_product msp on msp.id=msa.product_id
left join m_client cl on cl.id=msa.client_id
where st.is_reversed=0
AND msa.product_id = '36'
AND st.transaction_type_enum = 1
and LEFT(st.transaction_date, 4) = LEFT(NOW(), 4)
AND cl.id IN (
	SELECT DISTINCT msa.client_id
	FROM m_savings_account msa
	WHERE msa.product_id = '26'
)
GROUP BY 1
ORDER BY 1;
                ''', Mifosdb)
fd_corporate_mtd
load_table('gold','fd_corporate_mtd', fd_corporate_mtd)

In [0]:
%%time
if day == RUNDAY:
    fd_retail_mom = pd.read_sql(f'''
    SELECT LEFT(st.transaction_date, 7) AS Month, 
    sum(st.amount) AS Retail_FD_Amount,
    NOW() AS load_date
    from m_savings_account_transaction st
    inner join m_savings_account msa on msa.id =st.savings_account_id
    inner join m_savings_product msp on msp.id=msa.product_id
    left join m_client cl on cl.id=msa.client_id
    where st.is_reversed=0
    AND msa.product_id = '36'
    AND st.transaction_type_enum = 1
    and LEFT(st.transaction_date, 7) < LEFT(NOW(), 7)
    AND cl.id IN (
        SELECT DISTINCT msa.client_id
        FROM m_savings_account msa
        WHERE msa.product_id IN ('29', '31', '33')
    )
    GROUP BY 1
    ORDER BY 1;
                    ''', Mifosdb)
    fd_retail_mom

    load_table('gold','fd_retail_mom', fd_retail_mom)    
    
else:
    pass  

In [0]:
%%time
fd_retail_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(st.transaction_date, 10) AS DATE) AS Date, 
sum(st.amount) AS Retail_FD_Amount,
NOW() AS load_date
from m_savings_account_transaction st
inner join m_savings_account msa on msa.id =st.savings_account_id
inner join m_savings_product msp on msp.id=msa.product_id
left join m_client cl on cl.id=msa.client_id
where st.is_reversed=0
AND msa.product_id = '36'
AND st.transaction_type_enum = 1
and LEFT(st.transaction_date, 4) = LEFT(NOW(), 4)
AND cl.id IN (
	SELECT DISTINCT msa.client_id
	FROM m_savings_account msa
	WHERE msa.product_id IN ('29', '31', '33')
)
GROUP BY 1
ORDER BY 1;
                ''', Mifosdb)
fd_retail_mtd

load_table('gold','fd_retail_mtd', fd_retail_mtd)

In [0]:
%%time
if day == RUNDAY:
    account_product_trxn_mom = pd.read_sql(f'''
    select left(dat.transaction_date,7) as month, 
    dat.product_name, 
    isnull(count(case when channel = 'MOBILE' then dat.transaction_id end) ,0) as Mobile_Volume,
    isnull(count(case when channel = 'USSD' then dat.transaction_id end) ,0) as USSD_Volume,
    isnull(count(case when channel = 'CHATBOT' then dat.transaction_id end) ,0) as CHATBOT_Volume,
    isnull(SUM(case when channel = 'MOBILE' then dat.amount  end) ,0) as Mobile_Value,
    isnull(SUM(case when channel = 'USSD' then dat.amount  end) ,0) as USSD_Value,
    isnull(SUM(case when channel = 'CHATBOT' then dat.amount  end) ,0) as CHATBOT_Value,
    isnull(count(*) ,0) as Transaction_Count, 
    isnull(sum(dat.amount) ,0) as Transaction_Value,
    GETDATE() as load_date
    from dwh_all_transactions dat 
    where left(dat.transaction_date,7) < LEFT(GETDATE(), 7) 
    and dat.transaction_type  = 'withdrawal'
    and channel in ('MOBILE', 'USSD', 'CHATBOT')
    and response_code in ('00', '201', '200', '9000')
    group by 2,1
    order by 2, 1;
                    ''', conn)
    account_product_trxn_mom


    load_table('gold','account_product_trxn_mom', account_product_trxn_mom)    
    
else:
    pass 

In [0]:
%%time
account_product_trxn_mtd = pd.read_sql(f'''
select 
CAST(left(dat.transaction_date,10) AS DATE) as Date, 
dat.product_name, 
isnull(count(case when channel = 'MOBILE' then dat.transaction_id end) ,0) as Mobile_Volume,
isnull(count(case when channel = 'USSD' then dat.transaction_id end) ,0) as USSD_Volume,
isnull(count(case when channel = 'CHATBOT' then dat.transaction_id end) ,0) as CHATBOT_Volume,
isnull(SUM(case when channel = 'MOBILE' then dat.amount  end) ,0) as Mobile_Value,
isnull(SUM(case when channel = 'USSD' then dat.amount  end) ,0) as USSD_Value,
isnull(SUM(case when channel = 'CHATBOT' then dat.amount  end) ,0) as CHATBOT_Value,
isnull(count(*) ,0) as Transaction_Count, 
isnull(sum(dat.amount) ,0) as Transaction_Value,
GETDATE() as load_date
from dwh_all_transactions dat 
where left(dat.transaction_date,4) = LEFT(GETDATE(), 4) 
and dat.transaction_type  = 'withdrawal'
and channel in ('MOBILE', 'USSD', 'CHATBOT')
and response_code in ('00', '201', '200', '9000')
group by 2,1
order by 2,1;
                ''', conn)
account_product_trxn_mtd
load_table('gold','account_product_trxn_mtd', account_product_trxn_mtd)

In [0]:
%%time
if day == RUNDAY:
    product_liq_mom = pd.read_sql(f'''
    SELECT left(matt.transaction_date, 7) AS Month, 
    msp.name AS Product_Name, 
    COUNT(*) No_of_Liquidation,  
    SUM(msat.amount) AS Liquidation_Value, 
    NOW() AS load_date
    from m_account_transfer_transaction matt
    JOIN m_savings_account_transaction msat ON msat.id = matt.to_savings_transaction_id OR msat.id = matt.from_savings_transaction_id
    JOIN m_savings_account msa ON msa.id = msat.savings_account_id
    JOIN m_client mc ON mc.id = msa.client_id
    JOIN m_savings_product msp ON msp.id = msa.product_id
    WHERE left(matt.transaction_date, 7) < LEFT(NOW(), 7)
    AND msa.product_id IN ('36', '30')
    AND matt.description LIKE '%liq%'
    AND msat.transaction_type_enum ='2'
    GROUP BY 2, 1
    ORDER BY 2, 1;
                    ''', Mifosdb)
    product_liq_mom


    load_table('gold','product_liq_mom', product_liq_mom)    
    
else:
    pass  

In [0]:
%%time
product_liq_mtd = pd.read_sql(f'''
SELECT 
CAST(left(matt.transaction_date, 10) AS DATE) AS Date, 
msp.name AS Product_Name, 
COUNT(*) No_of_Liquidation,  
SUM(msat.amount) AS Liquidation_Value, 
NOW() AS load_date
from m_account_transfer_transaction matt
JOIN m_savings_account_transaction msat ON msat.id = matt.to_savings_transaction_id OR msat.id = matt.from_savings_transaction_id
JOIN m_savings_account msa ON msa.id = msat.savings_account_id
JOIN m_client mc ON mc.id = msa.client_id
JOIN m_savings_product msp ON msp.id = msa.product_id
WHERE left(matt.transaction_date, 4) = LEFT(NOW(), 4)
AND msa.product_id IN ('36', '30')
AND matt.description LIKE '%liq%'
AND msat.transaction_type_enum ='2'
GROUP BY 2, 1
ORDER BY 2, 1;
                ''', Mifosdb)
product_liq_mtd
load_table('gold','product_liq_mtd', product_liq_mtd)

In [0]:
%%time
if day == RUNDAY:
    cardless_mom = pd.read_sql(f'''
    SELECT 
    LEFT(MBS.POSTING_DATE, 7) AS Month,
    IFNULL(count(case when CHANNEL = 'MOBILE' then ID  END) ,0) as MOBILE_Volume,
    IFNULL(count(case when CHANNEL = 'USSD' then ID END) ,0) as USSD_Volume,
    IFNULL(count(case when CHANNEL = 'CHATBOT' then ID END) ,0) as CHATBOT_Volume,
    IFNULL(SUM(case when CHANNEL = 'MOBILE' then AMOUNT END) ,0) as MOBILE_Value,
    IFNULL(SUM(case when CHANNEL = 'USSD' then AMOUNT END) ,0) as USSD_Value,
    IFNULL(SUM(case when CHANNEL = 'CHATBOT' then AMOUNT END) ,0) as CHATBOT_Value,
    IFNULL(COUNT(*) ,0) as Total_Volume,
    IFNULL(sum(amount) ,0) as Total_Value,
    NOW() AS load_date
    FROM MM_BANK_SETTLEMENTS MBS
    WHERE LEFT(MBS.POSTING_DATE, 7) < LEFT(NOW(), 7)
    AND MBS.PARTNERS = 'PAYCODE'
    AND MBS.CHANNEL IN ('MOBILE', 'CHATBOT', 'USSD')
    AND MBS.RESPONSE_CODE = '201'
    GROUP BY 1
    ORDER BY 1;
                    ''', Dumpdb)
    cardless_mom


    load_table('gold','cardless_mom', cardless_mom)    
    
else:
    pass  

In [0]:
%%time
cardless_mtd = pd.read_sql(f'''
SELECT 
CAST(LEFT(MBS.POSTING_DATE, 10) AS DATE) AS Date,
IFNULL(count(case when CHANNEL = 'MOBILE' then ID  END) ,0) as MOBILE_Volume,
IFNULL(count(case when CHANNEL = 'USSD' then ID END) ,0) as USSD_Volume,
IFNULL(count(case when CHANNEL = 'CHATBOT' then ID END) ,0) as CHATBOT_Volume,
IFNULL(SUM(case when CHANNEL = 'MOBILE' then AMOUNT END) ,0) as MOBILE_Value,
IFNULL(SUM(case when CHANNEL = 'USSD' then AMOUNT END) ,0) as USSD_Value,
IFNULL(SUM(case when CHANNEL = 'CHATBOT' then AMOUNT END) ,0) as CHATBOT_Value,
IFNULL(COUNT(*) ,0) as Total_Volume,
IFNULL(sum(amount) ,0) as Total_Value,
NOW() AS load_date
FROM MM_BANK_SETTLEMENTS MBS
WHERE LEFT(MBS.POSTING_DATE, 4) = LEFT(NOW(), 4)
AND MBS.PARTNERS = 'PAYCODE'
AND MBS.CHANNEL IN ('MOBILE', 'CHATBOT', 'USSD')
AND MBS.RESPONSE_CODE = '201'
GROUP BY 1
ORDER BY 1;
                ''', Dumpdb)
cardless_mtd
load_table('gold','cardless_mtd', cardless_mtd)

In [0]:
%%time
if day == RUNDAY:
    retail_onboard_channel_mom = pd.read_sql(f'''
    SELECT LEFT(mc.activation_date, 7) AS MONTH,
    COUNT(case when mc.activatedon_userid = 31 then mc.id END) AS 'Mobile',
    COUNT(case when mc.activatedon_userid != 31 then mc.id END)  AS 'Others'
    FROM m_client mc
    LEFT JOIN m_savings_account msa ON msa.client_id = mc.id
    WHERE LEFT(mc.activation_date, 7 ) < LEFT(CURRENT_DATE(),7)
    AND msa.product_id IN ('29', '31', '33')
    GROUP BY 1
    ORDER BY 1;
                    ''', Mifosdb)
    retail_onboard_channel_mom


    load_table('gold','retail_onboard_channel_mom', retail_onboard_channel_mom)    
    
else:
    pass  

In [0]:
%%time
retail_onboard_channel_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(mc.activation_date, 10) AS DATE) AS Date,
COUNT(case when mc.activatedon_userid = 31 then mc.id END) AS 'Mobile',
COUNT(case when mc.activatedon_userid != 31 then mc.id END)  AS 'Others'
FROM m_client mc
LEFT JOIN m_savings_account msa ON msa.client_id = mc.id
WHERE LEFT(mc.activation_date, 4) = LEFT(CURRENT_DATE(),4)
AND msa.product_id IN ('29', '31', '33')
GROUP BY 1
ORDER BY 1;
                ''', Mifosdb)
retail_onboard_channel_mtd
load_table('gold','retail_onboard_channel_mtd', retail_onboard_channel_mtd)

In [0]:
%%time
accounts = pd.read_sql(f'''
SELECT mc.activation_date AS DATE, obd.BVN
FROM m_client mc
LEFT JOIN m_savings_account msa ON msa.client_id = mc.id
LEFT JOIN `Other Bank Details` obd ON obd.client_id = mc.id
WHERE LEFT(mc.activation_date, 4 ) = LEFT(CURRENT_DATE(),4)
AND LENGTH(obd.BVN) = 11 
AND msa.product_id IN ('29', '31', '33');
                ''', Mifosdb)
accounts
accounts['BVN2'] = "'" + accounts['BVN'] + "'"
str = ','.join([str(x) for x in accounts['BVN2'].tolist()])

gender = pd.read_sql(f'''
SELECT A.BVN_REGNO AS BVN, A.BVN_GENDER GENDER
FROM AM_VERIFIED_BVN A
WHERE A.BVN_REGNO IN ({str})
                ''', Dumpdb)
accounts = accounts.merge(gender, how='left', on="BVN")
accounts
del str

retail_gender_mtd = accounts.groupby(['DATE', 'GENDER']).size().reset_index(name='Count')
retail_gender_mtd

load_table('gold','retail_gender_mtd', retail_gender_mtd)

In [0]:
%%time
if day == RUNDAY:
    qr_code_mom = pd.read_sql(f'''
    SELECT LEFT(T.POSTING_DATE, 7)  AS MONTH,  T.QR_TYPE, T.TRANSACTION_STATUS,  COUNT(*) AS Volume, SUM(T.AMOUNT) AS Value, NOW() AS load_date
    FROM TM_QRCODE_TRANSACTIONS T
    WHERE LEFT(T.POSTING_DATE,  7) <  LEFT(CURRENT_DATE(),7)
    GROUP BY 1, 2, 3
    ORDER BY 1, 2, 3;

                    ''', Dumpdb)
    qr_code_mom.head()


    load_table('gold','qr_code_mom', qr_code_mom)    
    
else:
    pass  

In [0]:
%%time
qr_code_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(T.POSTING_DATE, 10) AS DATE) AS DATE,  T.QR_TYPE, T.TRANSACTION_STATUS,  COUNT(*) AS Volume, SUM(T.AMOUNT) AS Value, NOW() AS load_date
FROM TM_QRCODE_TRANSACTIONS T
WHERE LEFT(T.POSTING_DATE,  4) =  LEFT(CURRENT_DATE(),4)
GROUP BY 1, 2, 3
ORDER BY 1, 2, 3;

                ''', Dumpdb)
qr_code_mtd.head()
load_table('gold','qr_code_mtd', qr_code_mtd)

In [0]:
%%time
if day == RUNDAY:
    vpay_mom = pd.read_sql(f'''
    SELECT 
    left(tw.TIME, 7) AS MONTH,
    COUNT(case when tw.TRANSACTION_RESPONSE = '00' then tw.ID END) AS Vpay_Success_Volume,
    COUNT(case when tw.TRANSACTION_RESPONSE != '00' then tw.ID END) AS Vpay_Failed_Volume,
    SUM(case when tw.TRANSACTION_RESPONSE = '00' then tw.AMOUNT END) AS Vpay_Success_Value,
    SUM(case when tw.TRANSACTION_RESPONSE != '00' then tw.AMOUNT END) AS Vpay_Failed_Value,
    NOW() AS load_date
    FROM TM_WALLET_TRANSACTIONS tw
    WHERE tw.WALLET_NAME LIKE '%vpos%'
    AND left(tw.TIME, 7) < LEFT(NOW(), 7)
    GROUP BY 1
    ORDER BY 1;
    ''', Dumpdb)
    vpay_mom


    load_table('gold','vpay_mom', vpay_mom)    
    
else:
    pass  

In [0]:
%%time
vpay_mtd = pd.read_sql(f'''
SELECT 
CAST(left(tw.TIME, 10) AS DATE) AS DATE,
COUNT(case when tw.TRANSACTION_RESPONSE = '00' then tw.ID END) AS Vpay_Success_Volume,
COUNT(case when tw.TRANSACTION_RESPONSE != '00' then tw.ID END) AS Vpay_Failed_Volume,
SUM(case when tw.TRANSACTION_RESPONSE = '00' then tw.AMOUNT END) AS Vpay_Success_Value,
SUM(case when tw.TRANSACTION_RESPONSE != '00' then tw.AMOUNT END) AS Vpay_Failed_Value,
NOW() AS load_date
FROM TM_WALLET_TRANSACTIONS tw
WHERE tw.WALLET_NAME LIKE '%vpos%'
AND left(tw.TIME, 4) = LEFT(NOW(), 4)
GROUP BY 1
ORDER BY 1;
''', Dumpdb)
vpay_mtd
load_table('gold','vpay_mtd', vpay_mtd)

In [0]:
%%time

if day == RUNDAY:

    wallet_inter_intra_outflow_mom = pd.read_sql(f'''
    SELECT 
    LEFT(TWT.TIME,7) AS MONTH, 
    COUNT( CASE WHEN TWT.TO_BANK = '999999' then TWT.ID END) AS Intra_Volume,
    COUNT( CASE WHEN TWT.TO_BANK != '999999' then TWT.ID END) AS Inter_Volume,
    SUM( CASE WHEN TWT.TO_BANK = '999999' then TWT.AMOUNT END) AS Inter_Value,
    SUM( CASE WHEN TWT.TO_BANK != '999999' then TWT.AMOUNT END) AS Intra_Value,
    NOW() AS load_date
    FROM TM_WALLET_TRANSACTIONS TWT 
    WHERE LEFT(TWT.TIME,7) < LEFT(CURRENT_DATE(),7)
    AND TWT.TRANSACTION_RESPONSE = '00'
    AND TWT.TRANSACTION_TYPE = 'OUTFLOW'
    AND TWT.WALLET_NAME != 'Victor Nwaka'
    GROUP BY 1
    ORDER BY 1;
    ''', Dumpdb)
    wallet_inter_intra_outflow_mom.head(5)
    load_table('gold', 'wallet_inter_intra_outflow_mom', wallet_inter_intra_outflow_mom) 
    
else:
    pass  

In [0]:
%%time
wallet_inter_intra_outflow_mtd = pd.read_sql(f'''
SELECT 
CAST(LEFT(TWT.TIME,10) AS DATE) AS DATE, 
COUNT( CASE WHEN TWT.TO_BANK = '999999' then TWT.ID END) AS Intra_Volume,
COUNT( CASE WHEN TWT.TO_BANK != '999999' then TWT.ID END) AS Inter_Volume,
SUM( CASE WHEN TWT.TO_BANK = '999999' then TWT.AMOUNT END) AS Inter_Value,
SUM( CASE WHEN TWT.TO_BANK != '999999' then TWT.AMOUNT END) AS Intra_Value,
NOW() AS load_date
FROM TM_WALLET_TRANSACTIONS TWT 
WHERE LEFT(TWT.TIME,4) = LEFT(CURRENT_DATE(),4)
AND TWT.TRANSACTION_RESPONSE = '00'
AND TWT.TRANSACTION_TYPE = 'OUTFLOW'
AND TWT.WALLET_NAME != 'Victor Nwaka'
GROUP BY 1
ORDER BY 1;
''', Dumpdb)
wallet_inter_intra_outflow_mtd.head(5)
load_table('gold', 'wallet_inter_intra_outflow_mtd', wallet_inter_intra_outflow_mtd)

In [0]:
%%time
if day == RUNDAY:
    wallet_inter_intra_inflow_mom = pd.read_sql(f'''
    SELECT 
    LEFT(TWT.TIME,7) AS MONTH, 
    COUNT( CASE WHEN TWT.TO_BANK = '999999' then TWT.ID END) AS Intra_Volume,
    COUNT( CASE WHEN TWT.TO_BANK != '999999' then TWT.ID END) AS Inter_Volume,
    SUM( CASE WHEN TWT.TO_BANK = '999999' then TWT.AMOUNT END) AS Inter_Value,
    SUM( CASE WHEN TWT.TO_BANK != '999999' then TWT.AMOUNT END) AS Intra_Value,
    NOW() AS load_date
    FROM TM_WALLET_TRANSACTIONS TWT 
    WHERE LEFT(TWT.TIME,7) < LEFT(CURRENT_DATE(),7)
    AND TWT.TRANSACTION_RESPONSE = '00'
    AND TWT.TRANSACTION_TYPE = 'INFLOW'
    AND TWT.WALLET_NAME != 'Victor Nwaka'
    GROUP BY 1
    ORDER BY 1;
    ''', Dumpdb)
    wallet_inter_intra_inflow_mom.head(5)
    load_table('gold', 'wallet_inter_intra_inflow_mom', wallet_inter_intra_inflow_mom) 
    
else:
    pass  

In [0]:
%%time
wallet_inter_intra_inflow_mtd = pd.read_sql(f'''
SELECT 
CAST(LEFT(TWT.TIME,10) AS DATE) AS DATE, 
COUNT( CASE WHEN TWT.TO_BANK = '999999' then TWT.ID END) AS Intra_Volume,
COUNT( CASE WHEN TWT.TO_BANK != '999999' then TWT.ID END) AS Inter_Volume,
SUM( CASE WHEN TWT.TO_BANK = '999999' then TWT.AMOUNT END) AS Inter_Value,
SUM( CASE WHEN TWT.TO_BANK != '999999' then TWT.AMOUNT END) AS Intra_Value,
NOW() AS load_date
FROM TM_WALLET_TRANSACTIONS TWT 
WHERE LEFT(TWT.TIME,4) = LEFT(CURRENT_DATE(),4)
AND TWT.TRANSACTION_RESPONSE = '00'
AND TWT.TRANSACTION_TYPE = 'INFLOW'
AND TWT.WALLET_NAME != 'Victor Nwaka'
GROUP BY 1
ORDER BY 1;
''', Dumpdb)
wallet_inter_intra_inflow_mtd.head(5)
load_table('gold', 'wallet_inter_intra_inflow_mtd', wallet_inter_intra_inflow_mtd)

In [0]:
%%time
if day == RUNDAY:
    wallet_loans_mom = pd.read_sql(f'''
    SELECT LEFT(T.DATE_CREATED, 7) AS MONTH,
    COUNT(*) AS Loan_Volume,
    SUM(T.AMOUNT) AS Loan_Value,
    NOW() AS load_date
    FROM TM_WALLET_LOAN_TRANSACTIONS T
    WHERE LEFT(T.DATE_CREATED, 7) < LEFT(CURRENT_DATE(),7)
    AND T.`STATUS` = '00'
    GROUP BY 1
    ORDER BY 1;

    ''', Dumpdb)
    wallet_loans_mom.head(5)


    load_table('gold', 'wallet_loans_mom', wallet_loans_mom)  
    
else:
    pass  

In [0]:
%%time
wallet_loans_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(T.DATE_CREATED, 10) AS DATE) AS DATE,
COUNT(*) AS Loan_Volume,
SUM(T.AMOUNT) AS Loan_Value,
NOW() AS load_date
FROM TM_WALLET_LOAN_TRANSACTIONS T
WHERE LEFT(T.DATE_CREATED, 4) = LEFT(CURRENT_DATE(),4)
AND T.`STATUS` = '00'
GROUP BY 1
ORDER BY 1;

''', Dumpdb)
wallet_loans_mtd.head(5)
load_table('gold', 'wallet_loans_mtd', wallet_loans_mtd)

In [0]:
%%time
if day == RUNDAY:
    vbiz_dropoff_mom = pd.read_sql(f'''
    SELECT left(o.createdAt, 7) AS MONTH, 
    COUNT(o.id) AS No_of_Dropoffs,
    NOW() AS load_date
    FROM onboarding o
    WHERE left(o.createdAt, 7) < LEFT(CURRENT_DATE(), 7)
    AND o.nuban IS NULL
    GROUP BY 1
    ORDER BY 1;
    ''', vbizdb)
    vbiz_dropoff_mom.head(5)
    load_table('gold', 'vbiz_dropoff_mom', vbiz_dropoff_mom) 
    
else:
    pass  

In [0]:
%%time
vbiz_dropoff_mtd = pd.read_sql(f'''
SELECT CAST(left(o.createdAt, 10) AS DATE) AS DATE, 
COUNT(o.id) AS No_of_Dropoffs,
NOW() AS load_date
FROM onboarding o
WHERE left(o.createdAt, 4) = LEFT(CURRENT_DATE(), 4)
AND o.nuban IS NULL
GROUP BY 1
ORDER BY 1;
''', vbizdb)
vbiz_dropoff_mtd.head(5)
load_table('gold', 'vbiz_dropoff_mtd', vbiz_dropoff_mtd)

In [0]:
%%time
if day == RUNDAY:
    loan_disbursed_repaid_mom = pd.read_sql(f'''
    SELECT LEFT(ml.disbursedon_date,7) AS Disbursedon_Date,
    SUM(ml.principal_disbursed_derived) as Total_Principal_Disbursed, 
    SUM(ml.total_expected_repayment_derived) AS Total_Expected_Repayment, 
    SUM(ml.interest_charged_derived) AS Total_Interest_Charged,
    SUM(ml.principal_repaid_derived) AS Total_Principal_Repaid,
    NOW() AS load_date
    FROM m_loan ml 
    WHERE left(ml.disbursedon_date,7) < LEFT(NOW(),7)
    AND ml.disbursedon_date IS NOT NULL
    GROUP BY 1
    ORDER BY 1;
    ''', Mifosdb)
    loan_disbursed_repaid_mom.head()
    load_table('gold', 'loan_disbursed_repaid_mom', loan_disbursed_repaid_mom)
    
else:
    pass  

In [0]:
%%time
loan_disbursed_repaid_mtd = pd.read_sql(f'''
SELECT CAST(ml.disbursedon_date AS DATE) AS Disbursedon_Date,
SUM(ml.principal_disbursed_derived) as Total_Principal_Disbursed, 
SUM(ml.total_expected_repayment_derived) AS Total_Expected_Repayment, 
SUM(ml.interest_charged_derived) AS Total_Interest_Charged,
SUM(ml.principal_repaid_derived) AS Total_Principal_Repaid,
NOW() AS load_date
FROM m_loan ml 
WHERE left(ml.disbursedon_date,4) = LEFT(NOW(),4)
AND ml.disbursedon_date IS NOT NULL
GROUP BY 1
ORDER BY 1;
''', Mifosdb)
loan_disbursed_repaid_mtd.head()
load_table('gold', 'loan_disbursed_repaid_mtd', loan_disbursed_repaid_mtd)

In [0]:
%%time
if day == RUNDAY:
    sms_mom = pd.read_sql(f'''
    SELECT LEFT(cm.CREATED,7) AS MONTH,
    count(*) as COUNT, 
    sum(cm.HOLD_AMOUNT) as hold_amount,
    NOW() AS load_date
    FROM CM_NOTIFICATIONS_CHGS cm
    WHERE LEFT(cm.CREATED,7) < LEFT(NOW(),7)
    GROUP BY 1
    ORDER BY 1;
    ''', Dumpdb)
    sms_mom.head(5)
    load_table('gold', 'sms_mom', sms_mom)  
    
else:
    pass  

In [0]:
%%time
sms_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(cm.CREATED,10) AS DATE) AS DATE,
count(*) as COUNT, 
sum(cm.HOLD_AMOUNT) as hold_amount,
NOW() AS load_date
FROM CM_NOTIFICATIONS_CHGS cm
WHERE LEFT(cm.CREATED,4) = LEFT(NOW(),4)
GROUP BY 1
ORDER BY 1;
''', Dumpdb)
sms_mtd.head(5)
load_table('gold', 'sms_mtd', sms_mtd)

In [0]:
%%time
if day == RUNDAY:
    corporate_transaction_mom = pd.read_sql(f'''
    select LEFT(st.transaction_date,7)  AS Month,
    COUNT(case when st.transaction_type_enum = '2'  then st.id end) as 'Debit_volume',
    COUNT(case  when st.transaction_type_enum = '1' then st.id end) as 'Credit_volume',
    SUM(case when st.transaction_type_enum = '2'  then st.amount end) as 'Debit_Value',
    SUM(case  when st.transaction_type_enum = '1' then st.amount end) as 'Credit_Value',
    NOW() AS load_date
    from m_savings_account_transaction st
    inner join m_savings_account msa on msa.id =st.savings_account_id
    left join m_client cl on cl.id=msa.client_id
    where st.is_reversed=0
    AND LEFT(st.transaction_date,7) < LEFT(NOW(), 7)
    AND msa.product_id = '26'
    GROUP BY 1
    ORDER BY 1
    ''', Mifosdb)
    corporate_transaction_mom.head(5)
    load_table('gold', 'corporate_transaction_mom', corporate_transaction_mom)  
    
else:
    pass  

In [0]:
%%time
corporate_transaction_mtd = pd.read_sql(f'''
select CAST(LEFT(st.transaction_date,10) AS DATE)  AS DATE,
COUNT(case when st.transaction_type_enum = '2'  then st.id end) as 'Debit_volume',
COUNT(case  when st.transaction_type_enum = '1' then st.id end) as 'Credit_volume',
SUM(case when st.transaction_type_enum = '2'  then st.amount end) as 'Debit_Value',
SUM(case  when st.transaction_type_enum = '1' then st.amount end) as 'Credit_Value',
NOW() AS load_date
from m_savings_account_transaction st
inner join m_savings_account msa on msa.id =st.savings_account_id
left join m_client cl on cl.id=msa.client_id
where st.is_reversed=0
AND LEFT(st.transaction_date,4) = LEFT(NOW(), 4)
AND msa.product_id = '26'
GROUP BY 1
ORDER BY 1
''', Mifosdb)
corporate_transaction_mtd.head(5)
load_table('gold', 'corporate_transaction_mtd', corporate_transaction_mtd)

In [0]:
%%time
if day == RUNDAY:
    wallet_api_mom = pd.read_sql(f'''
    SELECT 
    LEFT(msc.DATE, 7) AS Month,
    msc.SUBSCRIBER_USERNAME AS subscriber,
    case when msc.API_RESOURCE_CONSUMED IN ('/bvn/basic', '/bvn/premium') AND msc.RESOURCE_VALUE LIKE '%/%' then 
      CONCAT(msc.API_RESOURCE_CONSUMED,'-account') ELSE msc.API_RESOURCE_CONSUMED END AS resource_consumed,
    COUNT(*) AS COUNT,
    NOW() AS load_date
    FROM MS_SUBSCRIBERS_CALLS msc 
    WHERE LEFT(msc.DATE, 7) < LEFT(NOW(), 7)
    and msc.SUBSCRIBER_USERNAME not in ('b.daniel','Victor Nwaka','v.nwaka','tunde_vfd_test')
    AND msc.RESPONSE_STATUS NOT IN ('Fail', 'Failed', 'FAIL', 'FAILED')
    GROUP BY 1,2,3
    ORDER BY 1,2,3;
    ''', Dumpdb)
    wallet_api_mom.head(20)
    load_table('gold', 'wallet_api_mom', wallet_api_mom)

    
else:
    pass  

In [0]:
%%time
wallet_api_mtd = pd.read_sql(f'''
SELECT 
CAST(LEFT(msc.DATE, 10) AS DATE) AS DATE,
msc.SUBSCRIBER_USERNAME AS subscriber,
case when msc.API_RESOURCE_CONSUMED IN ('/bvn/basic', '/bvn/premium') AND msc.RESOURCE_VALUE LIKE '%/%' then 
  CONCAT(msc.API_RESOURCE_CONSUMED,'-account') ELSE msc.API_RESOURCE_CONSUMED END AS resource_consumed,
COUNT(*) AS COUNT,
NOW() AS load_date
FROM MS_SUBSCRIBERS_CALLS msc 
WHERE LEFT(msc.DATE, 7) = LEFT(NOW(), 7)
and msc.SUBSCRIBER_USERNAME not in ('b.daniel','Victor Nwaka','v.nwaka','tunde_vfd_test')
AND msc.RESPONSE_STATUS NOT IN ('Fail', 'Failed', 'FAIL', 'FAILED')
GROUP BY 1,2,3
ORDER BY 1,2,3;
''', Dumpdb)
wallet_api_mtd.head()
load_table('gold', 'wallet_api_mtd', wallet_api_mtd)

In [0]:
%%time
if day == RUNDAY:
    vbiz_payment_mom = pd.read_sql(f'''
    SELECT LEFT(p.createdAt,7) AS MONTH,
    COUNT(*) AS Volume,
    SUM(p.amount) AS VALUE,
    NOW() AS load_date
    FROM payment p
    WHERE p.`status` = 'success'
    AND LEFT(p.createdAt,7) < LEFT(NOW(), 7)
    GROUP BY 1
    ORDER BY 1;

    ''', vbizdb)
    vbiz_payment_mom.head()
    load_table('gold', 'vbiz_payment_mom', vbiz_payment_mom)
    
else:
    pass  

In [0]:
%%time
vbiz_payment_mtd = pd.read_sql(f'''
SELECT CAST(LEFT(p.createdAt,10) AS DATE) AS DATE,
COUNT(*) AS Volume,
SUM(p.amount) AS VALUE,
NOW() AS load_date
FROM payment p
WHERE p.`status` = 'success'
AND LEFT(p.createdAt,4) = LEFT(NOW(), 4)
GROUP BY 1
ORDER BY 1;
''', vbizdb)
vbiz_payment_mtd.head()
load_table('gold', 'vbiz_payment_mtd', vbiz_payment_mtd)

In [0]:
%%time
if day == RUNDAY:
    nomiworld_e_voucher_mom = pd.read_sql(f'''
    select left(dat.transaction_date, 7) as month, 
    replace(replace(replace(dat.third_party_name, 'NOMIWORLD|',''), '|429', ''), '|133', '')  as e_voucher, 
    count(*) as Volume, 
    sum(dat.amount) as value,
    getdate() as load_date
    from dwh_all_transactions dat
    where left(dat.transaction_date, 7) < left(getdate(), 7)
    and dat.partners  = 'NOMIWORLD'
    and dat.response_code  = '0'
    group by 1,2
    order by 1,2;
    ''', conn)
    nomiworld_e_voucher_mom.head()
    load_table('gold', 'nomiworld_e_voucher_mom', nomiworld_e_voucher_mom) 
    
else:
    pass  

In [0]:
%%time
nomiworld_e_voucher_mtd = pd.read_sql(f'''
select CAST(left(dat.transaction_date, 10) AS DATE) AS DATE, 
replace(replace(replace(dat.third_party_name, 'NOMIWORLD|',''), '|429', ''), '|133', '')  as e_voucher, 
count(*) as Volume, 
sum(dat.amount) as value,
getdate() as load_date
from dwh_all_transactions dat
where left(dat.transaction_date, 4) = left(getdate(), 4)
and dat.partners  = 'NOMIWORLD'
and dat.response_code  = '0'
group by 1,2
order by 1,2;
''', conn)
nomiworld_e_voucher_mtd.head()
load_table('gold', 'nomiworld_e_voucher_mtd', nomiworld_e_voucher_mtd)

In [0]:
%%time
disbursed = pd.read_sql(f'''
SELECT CAST(ml.disbursedon_date AS DATE) AS date,
SUM(ml.principal_disbursed_derived) as Total_Disbursed
FROM m_loan ml 
WHERE left(ml.disbursedon_date,10) < CURDATE()
AND ml.disbursedon_date IS NOT NULL
GROUP BY 1
ORDER BY 1;
''', Mifosdb)
disbursed['date'] = pd.to_datetime(disbursed['date'])

expected = pd.read_sql(f'''
SELECT 
mlrs.duedate AS date,
SUM(mlrs.principal_amount + mlrs.interest_amount) AS Expected_Repayment_Amount
FROM m_loan_repayment_schedule mlrs
GROUP BY 1
ORDER BY 1;
''', Mifosdb)
expected['date'] = pd.to_datetime(expected['date'])


actual = pd.read_sql(f'''
SELECT 
mlt.transaction_date AS date,
SUM(mlt.amount) AS Actual_Repayment_Amount
FROM m_loan_transaction mlt
WHERE mlt.transaction_type_enum = 2
GROUP BY 1
ORDER BY 1;
''', Mifosdb)
actual['date'] = pd.to_datetime(actual['date'])

loan_disbursed_expected_actual_dod = df.merge(disbursed, how='left', on="date")
loan_disbursed_expected_actual_dod = loan_disbursed_expected_actual_dod.merge(expected, how='left', on="date")
loan_disbursed_expected_actual_dod = loan_disbursed_expected_actual_dod.merge(actual, how='left', on="date")
loan_disbursed_expected_actual_dod.fillna(0, inplace=True)
loan_disbursed_expected_actual_dod['load_date'] = datetime.datetime.now()
loan_disbursed_expected_actual_dod.head()

load_table('gold', 'loan_disbursed_expected_actual_dod', loan_disbursed_expected_actual_dod)